In [1]:
from io import open
import os
from conllu import parse
from tqdm import tqdm
import pandas as pd
import re

In [2]:
file = "Trains_93.CONLL"

In [3]:
data = """"""
max_count = 0
with open(file, 'r', encoding='utf-8') as read_file:
    for line in read_file:
        if line != "\n":
            data += line
        elif line == "\n":
            split_func = lambda line, i: line[i].split("|")
            sentences = parse(data,
                              fields=['id', 'form', 'lemma', 'upos', 'xpos', 'feats', 'head', 'deprel', 'deps',
                                      'misc', 'identity', 'bridging', 'discourse_deixis', 'reference', 'nom_sem'],
                              field_parsers={"identity": split_func, "nom_sem": split_func})

            sentence = sentences[0]

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(sentence)

In [ ]:
df = df[['form','identity']]
df.index = df.index + 1

In [ ]:
token_df = df.copy()

### index, identity

In [ ]:
identity_li = token_df.identity.tolist()

l_count = 0
r_count = 0

l_step_count = 0
r_step_count = 0

token_index = []
identities = []
temp_id = []
l_index = []

for index, identity in enumerate(identity_li):
    l_count += ''.join(identity).count('(')
    r_count += ''.join(identity).count(')')
    l_step_count = ''.join(identity).count('(')
    r_step_count += ''.join(identity).count(')')

    ### index processing###
    for i in range(l_step_count):
        l_index.append(index + 1)

    while (r_step_count):
        if l_index:
            l_idx = l_index.pop()
            token_index.append((l_idx, index + 1))
            r_step_count -= 1

    if l_count == r_count and identity == ['_']:
        l_count = 0
        r_count = 0
        continue

    temp_id += identity

    if l_count == r_count:  
        if l_count == 1:  # 중복 X
            identities.append(''.join(temp_id))
            temp_id = []

        else:  # 중복
            redundant_id = ''.join(temp_id)
            for i in range(l_count):
                temp_redundant_id = redundant_id[redundant_id.rindex('('):redundant_id.index(')') + 1]  # entity 안의 entity
                identities.append(temp_redundant_id)
                
                redundant_id = redundant_id.replace(temp_redundant_id, '')

            temp_id = []
            
        l_count = 0
        r_count = 0

In [ ]:
len(identities)
# len(token_index)
# token_index

In [ ]:
new_df = pd.DataFrame(columns=['entity', 'index', 'identity', 'head'])
new_df

In [ ]:
new_df['index'] = token_index
new_df['identity'] = identities
new_df.index = new_df.index+1
new_df.head(10)

### entity

In [ ]:
new_df['entity'] = ['0']*len(new_df)

In [ ]:
new_df.head(20)

In [ ]:
tokens = []
for index in token_index:
    temp_token=[]
    for idx in range(index[0]-1,index[-1]):
        temp_token.append(sentences[0][idx]['form'])
    tokens.append(' '.join(temp_token))

In [ ]:
new_df['entity'] = tokens
new_df.head(20)

In [ ]:
len(identities)

### head

In [ ]:
head_identity = []
pattern = re.compile(r"Min=\d+[,]{0,1}\d*")
for identity in identities:
    if identity == '':
        head_identity.append('')
    else:
        head= re.findall(pattern, identity)[0]
        head_id = head.replace("Min=","")
        head_identity.append(head_id)

In [ ]:
len(head_identity)

In [ ]:
new_df['head'] = head_identity
new_df.head(50)

### JSON 파일로 저장

In [ ]:
result_df = new_df.copy()
result_df

In [ ]:
result_df = result_df[['entity','index','head']]
result_df

In [ ]:
name_change_idx = file.index('.')
name_change = file[name_change_idx:]
name_change
file_json = file.replace(name_change,'_json')
file_json

In [ ]:
import json

In [ ]:
result = result_df.to_json(orient='columns')
parsed = json.loads(result)

with open(file_json, 'w') as outfile:
    json.dump(parsed, outfile, indent=4)